<center>

<h1> Control in Prolog </h1>
<h2> CS3100 Monsoon 2020 </h2>
</center>

## Review

### Previously

* Programming with Lists, Arithmetic, Backtracking, Choice Points

### This lecture

* Control in Prolog
  + Rule order and Goal order
  + An abstract interpreter for logic programs
    * Unification, Substitution

## Algorithm = Logic + Control

* Logic: facts, rules and queries
* Control: how prolog chooses the rules and goals, among several available options.

There are two main control decisions: **Rule Order & Goal Order.**

## Algorithm = Logic + Control

* **Rule order**: Given a program with a collection of facts and rules, in which order do you choose to pick rule to unify.
  + SWI-Prolog chooses the **first** applicable rule in the order in which they appear in the program.
* **Goal order**: Given a set of goals to resolve, which goal do you choose
  + SWI-Prolog: chooses the **left-most** subgoal. 
  
Rule order and goal order influences program behaviour. 

## Substring in Prolog

```
<-------X-------->
+-------------------------------+
|      |    S    |              |  
+-------------------------------+
<--------------Z---------------->
```

We can specify `S` this is seemingly equivalent ways.

* prefix X of Z and suffix S of X.
* suffix S of X and prefix X of Z.

## Substring in Prolog

The corresponding prolog queries are:

In [1]:
append([],Q,Q).
append([H | P], Q, [H | R]) :- append(P, Q, R).
prefix(X,Z) :- append(X,Y,Z).
suffix(Y,Z) :- append(X,Y,Z).

Added 4 clauses(s).

## Substring in Prolog

They usually produce the same result:

In [2]:
?- prefix([a,b,c],[a,b,c,d]), suffix(S,[a,b,c]).

S = [ a, b, c ] ;
S = [ b, c ] ;
S = [ c ] ;
S = [  ] .

In [3]:
?- suffix(S,[a,b,c]), prefix([a,b,c],[a,b,c,d]).

S = [ a, b, c ] ;
S = [ b, c ] ;
S = [ c ] ;
S = [  ] .

## Substring in Prolog

Their answers however differ in other cases:

In [4]:
?- prefix(X,[b]), suffix([a],X).

false.

In [5]:
?- suffix([a],X), prefix(X,[b]).

: 

## Goal order changes solutions

Consider the query:

In [6]:
?- suffix([a],L), prefix(L,[a,b,c]) {1}.

L = [ a ] .

In [7]:
?- suffix([a],L), prefix(L,[a,b,c]) {2}.

: 

**Exercise**: Trace by hand.

## Goal order changes solutions

Consider the query:

In [8]:
?- prefix(L,[a,b,c]), suffix([a],L).

L = [ a ] .

has precisely one answer.

**Exercise**: Trace by hand.

## Rule order affects the search for solutions

Consider the definition `appen2` which reorders the rules from `append`.

In [9]:
appen2([H | P], Q, [H | R]) :- appen2(P, Q, R).
appen2([],Q,Q).

Added 2 clauses(s).

## Rule order affects the search for solutions

Consider the query:

In [10]:
?- append(X,[c],Z) {5}.

X = [  ], Z = [ c ] ;
X = [ _226 ], Z = [ _226, c ] ;
X = [ _226, _238 ], Z = [ _226, _238, c ] ;
X = [ _226, _238, _250 ], Z = [ _226, _238, _250, c ] ;
X = [ _226, _238, _250, _262 ], Z = [ _226, _238, _250, _262, c ] .

In [11]:
?- appen2(X,[c],Z) {1}.

: 

goes down an infinite search path.

**Exercise**: Trace by hand.

## Occurs check problem

Consider the query

In [ ]:
?- append([],E,[a,b|E]).

goes down an infinite search path. (Must restart kernel).

**Exercise:** Trace by hand to verify why.

## Occurs check problem

Consider the query

```
?- append([],E,[a,b | E]).
```

* In order to unify this with, `append([],Y,Y)`, we will unify `E = [a,b | E]`, whose solution is `E = [a,b,a,b,a,b,...]`.
* In the name of efficiency, most prolog implementations do not check whether `E` appears on the RHS term.
  + infinite loop on unification.
  + The efficacy of this optimization in 2020 is dubious.
* Some versions of prolog creates cyclic terms (like OCaml recursive values), but most don't. 

## Occurs check problem

You can explicitly turn on occurs check in SWI Prolog. 

In [1]:
?- set_prolog_flag(occurs_check,true).

true.

In [2]:
?- append([],E,[a,b | E]).

false.

## Occurs check problem

You can explicitly turn occurs check in SWI Prolog to an **error**.

In [3]:
?- set_prolog_flag(occurs_check,error).

true.

In [4]:
?- append([],E,[a,b | E]).

: 

## Abstract interpreter for logic programs

We can precisely define the influence of rule and goal orders by describing an **abstract interpreter** for logic programs.

First, we will start off with some definitions of ideas that we have informally seen earlier. 

## Substitution

A substitution is a finite set of pairs of terms $\{X_1/t_1, \ldots, X_n/t_n\}$ where each $t_i$ is a term and each $X_i$ is a variable such that $X_i \neq t_i$ and $X_i \neq X_j$ if $i \neq j$.

The empty substitution is denoted by $\epsilon$.

For example, $\sigma = \{X/[1,2,3], Y/Z, Z/f(a,b)\}$ is valid substitution.

## Quiz

Is this a valid substitution? 

\\[
\{X/Y,Y/X,Z/Z,A/a1,A/a2,m/n\}
\\]

## Quiz

Is this a valid substitution? 

\\[
\sigma = \{X/Y,Y/X,Z/Z,A/a1,A/a2,m/n\}
\\]

No. 

* $Z/Z$ should not be in $\sigma$.
* Variable $A$ has two substitutions $A/a1$ and $A/a2$, which is incorrect.
* $m/n$ is not a valid substitution; $m$ should be a variable.
* $X/Y,Y/X \in \sigma$ is fine. 

## Application of substitution

The application of substitution $\sigma$ to a variable $X$, written as $X\sigma$ is defined 

\\[
X\sigma = 
\begin{cases}
t \text{ if } X/t \in \sigma \\
X \text{ otherwise}
\end{cases}
\\]

## Application of substitution

Let $\sigma$ be a substitution $\{X_1/t_1, \ldots, X_n/t_n\}$ and $E$ a term or a formula. The application $E\sigma$ of $\sigma$ to $E$ is obtained by **simultaneously** replacing every occurrence of $X_i$ in E with $t_i$. 

Given $\sigma = \{X/[1,2,3], Y/Z, Z/f(a,b)\}$ and $E = f(X,Y,Z)$, $E\sigma = f([1,2,3],Z,f(a,b))$.

Now, $E\sigma$ is known as an **instance** of $E$.

## Composition of substitutions

Consider two substitutions $\theta$ and $\sigma$. Then, the composition is defined as $\theta\sigma$. Intuitively, we will apply the substitution $\theta$ before $\sigma$ in $\theta\sigma$. 

Consider $\theta = \{X/Y,Z/a\}$ and $\sigma = \{Y/X,Z/b\}$. Then, $\theta\sigma = \{Y/X,Z/a\}$.

Let $\theta = \{X_1/s_1, \ldots, X_n/s_n\}$ and $\sigma = \{Y_1/t_1, \ldots, Y_n/t_n\}$ be two substitutions. The composition $\theta\sigma$ is the set obtained from the set:

\\[
\{X_1/(s_1\sigma), \ldots, X_n/(s_n\sigma),Y_1/t_1,\ldots,Y_n/t_n\}
\\]

1. and removing all $~X_i/s_i\sigma~$ for which $X_i$ is syntactically equal to $s_i\sigma$ and 
2. and removing those $Y_i/t_i$ for which $Y_i$ is the same as some $X_j$.



## Properties of substitutions

Let $\theta$, $\sigma$ and $\gamma$ be substitutions, $\epsilon$ be empty substitution, and let $E$ by a term or a formula. Then:

* $E(\theta\sigma) = (E\theta)\sigma$
* $(\theta\sigma)\gamma = \theta(\sigma\gamma)$
* $\epsilon\theta = \theta\epsilon = \theta$.
* $\theta = \theta\theta$ iff $Dom(\theta) \cap Range(\theta) = \emptyset$.

For example, let $\theta = \{X/Y,Y/X\}$. Then, $\theta\theta = \epsilon \neq \theta$.

## Properties of substitutions

In general, composition of substitutions is not commutative. For example,

\\[
\{X/f(Y)\}\{Y/a\} = \{X/f(a),Y/a\} \neq \{Y/a\}\{X/f(Y)\} = \{Y/a,X/f(Y)\}
\\]

## Unifier

Let $s$ and $t$ be two terms. A substitution $\sigma$ is a unfier for $s$ and $t$ if $s\sigma$ and $t\sigma$ are syntactically equal. 

Let $s = f(X,Y)$ and $t = f(g(Z),Z)$. Let $\sigma = \{X/g(Z), Y/Z\}$ and $\theta = \{X/g(a),Y/a,Z/a\}$. Both $\sigma$ and $\theta$ are unifiers for $s$ and $t$. 

A substitution is $\sigma$ is more general than another substitution $\theta$ if there exists a substitution $\omega$ such that $\theta = \sigma\omega$.

In the previous example, $\theta = \sigma\{Z/a\}$. Hence, $\sigma$ is more general than $\theta$.

## Most general unifier

A unfier is said to be the most general unfier (mgu) of two terms if it is more general than any other unfier of the terms.

A pair of terms may have more than one most general unifier. For example, for the terms $f(X,X)$ and $f(Y,Z)$, the unifiers $\theta = \{X/Y,Z/Y\}$ and $\sigma = \{X/Z,Y/Z\}$ are both most general unifier. 

$\theta = \sigma\{Z/Y\}$ and $\sigma = \theta\{Y/Z\}$.

If the unfiers $\theta$ and $\sigma$ are both mgus, then there is a substitution $\gamma = \{X_1/Y_1,\ldots,X_n/Y_n\}$ where $X_i$ and $Y_i$ are variables such that $\theta = \sigma\gamma$.

Intuitively, $\theta$ can be obtained from $\sigma$ by **renaming** the variables.

## Quiz

What is the mgu of $f(X,X,Y)$ and $f(Y,Z,a)$

1. $\{X/a,Y/a,Z/a\}$
2. $\{X/b,Y/b,Z/b\}$
3. $\{X/Y,Z/Y\}$
4. $\epsilon$

## Quiz

What is the mgu of $f(X,X,Y)$ and $f(Y,Z,a)$

1. $\{X/a,Y/a,Z/a\}$ **✓**
2. $\{X/b,Y/b,Z/b\}$
3. $\{X/Y,Z/Y\}$
4. $\epsilon$

## Algorithm for computing mgu

**Input:** Two terms $T_1$ and $T_2$ to be unified and the mgu $\theta$.

**Output:** $\theta$, the mgu if $T_1$ and $T_2$ or raises *FAIL* exception.

**Algorithm:** $mgu(T_1,T_2)$.

```
mgu(X,Y,𝜃) = 
  X = X𝜃
  Y = Y𝜃
  case 
    X is a variable that does not occur in Y:
      return (𝜃{X/Y} ∪ {X/Y})
    Y is a variable that does not occur in X:
      return (𝜃{Y/X} ∪ {Y/X})
    X and Y are indentical constants or variables:
      return 𝜃
    X is f(X1,...,Xn) and Y is f(Y1,...,Yn):
      return (fold (fun 𝜃 (X,Y) -> mgu(X,Y,𝜃)) 𝜃 [(X1,Y1);...;(Xn,Yn)])
    otherwise:
      raise FAIL
}

let mgu(X,Y) = mgu(X,Y,ϵ)
```

## Trace

\\[
\begin{array}{rl}
  & mgu(f(X,X,Y),f(Y,Z,a)) \\
= & mgu(f(X,X,Y),f(Y,Z,a),\epsilon) \\
= & \text{fold} ~(\text{fun} ~\theta ~(X,Y) \rightarrow mgu(X,Y,\theta)) ~\epsilon ~[(X,Y);(X,Z);(Y,a)] \\
  & \quad mgu(X,Y,\epsilon) = \\
  & \quad \quad X\epsilon = X, Y\epsilon = Y \\
  & \quad \quad \epsilon\{X/Y\} \cup \{X/Y\} = \{X/Y\} \\
= & \text{fold} ~(\text{fun} ~\theta ~(X,Y) \rightarrow mgu(X,Y,\theta)) ~\{X/Y\} ~[(X,Z);(Y,a)] \\
  & \quad mgu(X,Z,\{X/Y\}) = \\
  & \quad \quad X\{X/Y\} = Y, Z\{X/Y\} = Z \\
  & \quad \quad \{X/Y\}\{Y/Z\} \cup \{Y/Z\} = \{X/Z,Y/Z\} \\
= & \text{fold} ~(\text{fun} ~\theta ~(X,Y) \rightarrow mgu(X,Y,\theta)) ~\{X/Z,Y/Z\} ~[(Y,a)] \\
  & \quad mgu(Y,a,\{X/Z,Y/Z\}) = \\
  & \quad \quad Y\{X/Z,Y/Z\} = Z, a\{X/Z,Y/Z\} = a \\
  & \quad \quad \{X/Z,Y/Z\}\{Z/a\} \cup \{Z/a\} = \{X/a,Y/a,Z/a\} \\
= & \{X/a,Y/a,Z/a\}\\
\end{array}
\\]

## Abstract interpreter

**Input**: A goal G and a program P

**Output:** An instance of G that is a logical consequence of P, or **false** otherwise. 

**Algorithm:** run(P,G)

**Note:** _Resolvant_ is list of goals.

```
Initialise resolvent to G.
while (the resolvent is not empty) {
  choose a goal A from the resolvent //goal order
  choose a (renamed) clause A' <- B1,...,Bn from P //rule order
      such that A and A' unify with mgu 𝜃
    (if no such goal and clause exist, exit the while loop).
  replace A by B1,...,Bn in the resolvent
  apply 𝜃 to the resolvent and G
}
If the resolvent is empty, then output G, else output false. 
```

## Abstract interpreter is non-deterministic

* Observe that the abstract interpreter deliberately does not encode the rule and goal order explicitly.
  + Nor does it encode what happens on failure to prove a goal (no backtracking).
  + How does it work then?

Consider the program:

```
plus(1,3,4).
plus(2,2,4).
even(2).
```

and the goal `plus(X,Y,4), even(X)`.

* Both `plus(2,2,4)` and `plus(1,3,4)` unify with `plus(X,Y,4)`. 
* But only `plus(2,2,4)` ensures that the second goal `even(X)` is satisfied. 
* Since the abstract interpreter in non-deterministic, one of its behaviours is to choose `plus(2,2,4)`, which will lead to success without failure.

## Abstract interpreter is non-deterministic

Consider the program:

```
plus(1,3,4).
plus(2,2,4).
even(2).
```

and the goal `plus(X,Y,4), even(X)`.

OTOH, if the second goal `even(X)` is chosen as the first to resolve, then it will only unify with `even(2)`, which will change the other goal to `plus(2,Y,4)` which leaves only one choice.

**Non-determinism is essential for correctness of abstract interpreter.**

## Backtracking and choice points

* The abstract interpreter does not explicitly encode backtracking (recover from bad choices) and choice points (present more than one result).
* The program is said to be **deterministic**, if there is exactly one clause from the program to reduce each goal.
  + No backtracking and choice points are necessary. 

## Backtracking and choice points - Assignment 7

* Backtracking and choicepoints are encoded more naturally in a recursive formulation of the abstract interpreter.
  + Left as an exercise to you.
  + `List.fold_left` is your friend.
* You will implement an interpreter for prolog in OCaml with backtracking and choice points for Assignment 7.
  + 1/3 of the score in assignment 7 will test backtracking and choice point implementation of your interpreter. 


<center>

<h1> Fin. </h1>
</center>